# Trabalho 3
## Lógica Computacional 2020-2021

> O objetivo deste trabalho é a utilização do sistema Z3 na análise de propriedades temporais de sistemas dinâmicos modelados por  FOTS (“First Order Transition Systems”) .

> Trabalho realizado por: 
    > 1. Paulo Costa - A87986
    > 2. André Araújo - A87987

### Exercício 1
1. O seguinte sistema dinâmico denota 4 inversores ($\,A, B, C, D\,$) que lêm um bit num canal input e escrevem num canal output uma transformação desse bit.
> i. Cada inversor tem um bit $s$ de estado, inicializado  com um valor aleatório. \
> ii. Cada inversor é regido pelas seguintes transformações    
                          $$\mathbf{invert}\mathtt(in,out)$$   
                          $$x \gets \mathsf{read}(\mathtt{in})$$  
                          $$s \gets \neg x\;\;\|\;\; s\gets s$$              
                          $$\mathsf{write}(\mathtt{out},s)$$                         
> iii. O sistema termina quando todos os inversores tiverem o estado $\,s=0\,$. 

   


> a. Construa um FOTS que descreva este sistema e implemente este sistema, numa abordagem BMC (“bouded model checker”) num traço com $\,n\,$ estados. \
  b. Verifique usando $k$-lookahead se o sistema  termina \
        ou, em alternativa, \
  c. Explore as técnicas que estudou para verificar em que condições o sistema termina.

## Resolução:

a-
Neste caso o estado do FOTS respectivo serão 5 de inteiros, o primeiro contendo o valor do $\mathit{pc}$ (o *program counter* que neste caso pode ser binario  0 ou 1) e os restantes os valores das variáveis $s$. O estado inicial é caracterizado pelo seguinte predicado:

$$
\mathit{pc} = 0\wedge(s_A = 1 \vee s_A = 0)\wedge(s_B = 1 \vee s_B = 0)\wedge (s_D = 1 \vee s_D = 0)\wedge (s_C = 1 \vee s_C = 0)
$$

As transições possíveis no FOTS são caracterizadas pelo seguinte predicado:

$$
\begin{array}{c}
(\mathit{pc} = 0 \wedge s_A = 0  \wedge s_B = 0 \wedge s_D = 0 \wedge s_C = 0 \wedge \\ 
\mathit{pc}' = 1 \wedge s_A' = s_A \wedge s_B' = s_B \wedge s_D' = s_D \wedge s_C' = s_C))\\ 
\vee\\
(\mathit{pc} = 0 \wedge \mathit{pc}' = 0 \wedge(s_A = 1 \vee s_A = 0)\wedge\\ 
(s_B = 1 \vee s_B = 0)\wedge (s_D = 1 \vee s_D = 0)\wedge (s_C = 1 \vee s_C = 0) \wedge \\
(s_A' = s_A \vee s_A'=\neg s_C )\wedge (s_B' = s_B \vee s_B'=\neg s_A' )\wedge (s_D' = s_D \vee s_D'=\neg s_B' )\wedge (s_C' = s_C \vee s_C'=\neg s_D' )\\
\vee\\
(\mathit{pc} = 1 \wedge \mathit{pc}' = 1 \wedge s_A' = s_A \wedge s_B' = s_B \wedge s_D' = s_D \wedge s_C' = s_C)
\end{array}
$$


> Note que este predicado é uma disjunção de todas as possíveis transições que podem ocorrer no programa. Cada transição é caracterizada por um predicado onde uma variável do programa denota o seu valor no pré-estado e a mesma variável com apóstrofe denota o seu valor no pós-estado.

> Este procedimento designa-se model checking e, quando uma propriedade não é válida, produz um contra-exemplo (um traço do FOTS correspondente a uma execução do programa onde a propriedade falha). Bounded model checking é uma técnica particular de model checking, onde o objectivo é determinar se uma propriedade temporal é válida nos primeiros  𝐾  estados da execução do FOTS.

>Definimos então a função $declare$, seguida da função $init$ e da $trans$ e finalmente a função $gera$_$traco$.

In [7]:
from z3 import *

def declare(i): #declara as variaveis de cada estado
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['s_A'] = Int('s_A'+str(i))
    state['s_B'] = Int('s_B'+str(i))
    state['s_D'] = Int('s_D'+str(i))
    state['s_C'] = Int('s_C'+str(i))
    return state

def init(state): #inicia o primeiro estado 
    return And(Or(state['s_A']==0,state['s_A']==1),Or(state['s_B']==0,state['s_B']==1),
               Or(state['s_D']==0,state['s_D']==1),Or(state['s_C']==0,state['s_C']==1)
               ,state['pc']==0)

In [8]:
def trans(curr,prox): #define as tranciçoes possiveis
    # delimita os valores do proximo estado para as variaveis 
    tl=And(Or(prox['s_A']==0,prox['s_A']==1),Or(prox['s_B']==0,prox['s_B']==1),
               Or(prox['s_D']==0,prox['s_D']==1),Or(prox['s_C']==0,prox['s_C']==1))
    # define igualdade dos valores do estado anterior para o seguinte
    ti=And(prox['s_A']==curr['s_A'],prox['s_B']==curr['s_B'],
           prox['s_D']==curr['s_D'],prox['s_C']==curr['s_C'])
    # operaçoes para quando nao alcança o objetivo 
    A=Or(prox['s_A']==curr['s_A'],prox['s_A']!=curr['s_C'])
    B=Or(prox['s_B']==curr['s_B'],prox['s_B']!=prox['s_A'])
    D=Or(prox['s_D']==curr['s_D'],prox['s_D']!=prox['s_B'])
    C=Or(prox['s_C']==curr['s_C'],prox['s_C']!=prox['s_D'])
    
    # as possiveis tranciçoes
    t1=And(curr['pc']==0,prox['pc']==1,curr['s_A']==0,curr['s_B']==0,curr['s_D']==0,curr['s_C']==0,ti)
    t2=And(curr['pc']==0,prox['pc']==0,A,B,D,C,tl,Or(curr['s_A']==1,curr['s_B']==1,curr['s_D']==1,curr['s_C']==1),Or(prox['s_A']!=curr['s_A'],prox['s_B']!=curr['s_B'],prox['s_D']!=curr['s_D'],prox['s_C']!=curr['s_C']))
    t3=And(curr['pc']==1,prox['pc']==1,ti)
    return Or(t1,t2,t3)

In [9]:
def gera_traco(declare,init,trans,k):
    s = Solver()
    state =[declare(i) for i in range(k)]
    s.add(init(state[0]))
    for i in range(k-1):
        s.add(trans(state[i],state[i+1]))
    if s.check()==sat:
        m=s.model()
        for i in range(k):
            print(i)
            for x in state[i]:
                print(x,"=",m[state[i][x]])
        

gera_traco(declare,init,trans,50)

0
pc = 0
s_A = 0
s_B = 1
s_D = 1
s_C = 1
1
pc = 0
s_A = 0
s_B = 1
s_D = 1
s_C = 0
2
pc = 0
s_A = 1
s_B = 1
s_D = 1
s_C = 0
3
pc = 0
s_A = 1
s_B = 1
s_D = 0
s_C = 1
4
pc = 0
s_A = 1
s_B = 0
s_D = 1
s_C = 1
5
pc = 0
s_A = 0
s_B = 0
s_D = 1
s_C = 1
6
pc = 0
s_A = 0
s_B = 0
s_D = 1
s_C = 0
7
pc = 0
s_A = 0
s_B = 1
s_D = 1
s_C = 0
8
pc = 0
s_A = 0
s_B = 1
s_D = 0
s_C = 0
9
pc = 0
s_A = 1
s_B = 1
s_D = 0
s_C = 1
10
pc = 0
s_A = 1
s_B = 0
s_D = 1
s_C = 1
11
pc = 0
s_A = 0
s_B = 1
s_D = 1
s_C = 0
12
pc = 0
s_A = 1
s_B = 1
s_D = 1
s_C = 0
13
pc = 0
s_A = 1
s_B = 1
s_D = 0
s_C = 0
14
pc = 0
s_A = 1
s_B = 0
s_D = 0
s_C = 1
15
pc = 0
s_A = 1
s_B = 0
s_D = 1
s_C = 1
16
pc = 0
s_A = 0
s_B = 1
s_D = 1
s_C = 0
17
pc = 0
s_A = 1
s_B = 1
s_D = 0
s_C = 0
18
pc = 0
s_A = 1
s_B = 0
s_D = 0
s_C = 1
19
pc = 0
s_A = 1
s_B = 0
s_D = 1
s_C = 1
20
pc = 0
s_A = 0
s_B = 0
s_D = 1
s_C = 0
21
pc = 0
s_A = 0
s_B = 1
s_D = 1
s_C = 0
22
pc = 0
s_A = 1
s_B = 1
s_D = 0
s_C = 1
23
pc = 0
s_A = 1
s_B = 0
s_D = 1
s_C = 1
24

### Conclusão (a):

>Bem, achamos que a primeira alínea deste exercício foi realizada com bastante sucesso. Acabamos por conseguir pôr em prática mais uma vez, os conhecimentos teóricos adquiridos na disciplina e acabar por entendê-los de uma forma bastante interessante. \
\
>Utilizando a função $declare$ declara as variáveis de estado, agrupadas num dicionário que nos permite aceder às mesmas pelo nome.\
A função $init$ inicia o primeiro estado a utilizar.
Em seguida, a função $trans$ permite-nos definir as transições possíveis, ou seja, testa se é possível transitar de um estado para o seguinte, através das condições colocadas. \
E por fim, a função $gera$_$traco$ que gera uma cópia das variáveis do estado, testando condicões impostas, tais como: se um estado é inicial e se um par de estados é uma transição válida. \
\
> Assim foi-nos possível realizar esta alínea com sucesso e realizando os objetivos pretendidos.
                               

## Resolução:

> c) \
Aqui tivemos várias "étapas", que vamos passar a explicar, antes de apresentarmos cada uma das funções respetivas. \
Primeiramente, para provar que a transiçao acaba temos de verificar se em algum estado o pc (program counter) chega a ter valor 1, e portanto definimos a função $termina$:


In [10]:
def termina(state):
    return state['pc'] ==1

> Em seguida, definimos a função $bmc$_$eventually$, esta que tem como objetivo verificar se de facto, o programa termina.

In [11]:
def bmc_eventually(declare,init,trans,prop,bound):
    for k in range(1,bound+1):
        s = Solver()
        state =[declare(i) for i in range(k)]
        s.add(init(state[0]))
        for i in range(k-1):
            s.add(trans(state[i],state[i+1]))
        s.add(prop(state[k-1]))
        if s.check()==sat:
            m=s.model()
            for i in range(k):
                print(i)
                for x in state[i]:
                    print(x,"=",m[state[i][x]])
            return
    print ("Não foi possivel verificar a proposicao com "+str(bound)+' tracos')


bmc_eventually(declare,init,trans,termina,20)

0
pc = 0
s_A = 0
s_B = 0
s_D = 0
s_C = 0
1
pc = 1
s_A = 0
s_B = 0
s_D = 0
s_C = 0


> Agora, tendo verificado que o programa termina quando todos começam a 0, tivemos de verificar se ele termina quando pelo menos um deles começa a 1, ou seja:
$$ S_A=1\vee S_B=1\vee S_D=1\vee S_C=1$$
E portanto, definimos a função $bmc$_$eventually$_$not0$, que nos torna possível esta verificação necessária para a conclusão do exercício. 

In [12]:
def bmc_eventually_not0(declare,init,trans,prop,bound):
    for k in range(1,bound+1):
        s = Solver()
        state =[declare(i) for i in range(k)]
        s.add(init(state[0]))
        s.add(Or(state[0]['s_A']==1,state[0]['s_B']==1,state[0]['s_D']==1,state[0]['s_C']==1))
        for i in range(k-1):
            s.add(trans(state[i],state[i+1]))
        s.add(prop(state[k-1]))
        if s.check()==sat:
            m=s.model()
            for i in range(k):
                print(i)
                for x in state[i]:
                    print(x,"=",m[state[i][x]])
            return
    print ("Não foi possivel verificar a proposicao com "+str(bound)+' tracos')

bmc_eventually_not0(declare,init,trans,termina,50)

Não foi possivel verificar a proposicao com 50 tracos


### Conclusão (c):

> Decidimos resolver pela alternativa, da alínea (c), e achamos que acabamos por conseguir apresentar o objetivo pretendido. \
Conseguimos mais uma vez, colocar em prática os conhecimentos obtidos nas aulas e desta vez, achamos interessante o facto de termos de escolher perante duas opções, isto porque nos deixou a pensar em ambas as formas e optar por apresentar uma delas. \
Achamos que esta alínea acabou por ser completa da melhor forma.

> Em jeito de conclusão, esperemos que tenhamos realizado o exercício da forma que o professor pretendia e que de facto, tenhamos cumprido todos os requisitos impostos, algo que achamos que foi cumprido!